# Проект "Отток клиентов"

## Описание проекта

Из «Бета-Банка» стали уходить клиенты. Маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых. Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. Нужно построить модель с предельно большим значением F1-меры на тестовой выборке (не менее 0.59). Дополнительно нужно измерить AUC-ROC, сравнить её значение с F1-мерой.

## План выполнения проекта

1. [**Загрузите и подготовьте данные.**](#step1) Поясните порядок действий. Путь к файлу: <i>/datasets/Churn.csv</i>.
1. [**Исследуйте баланс классов, обучите модель без учёта дисбаланса.**](#step2) Кратко опишите выводы.
1. [**Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую.**](#step3) Кратко опишите выводы.
1. [**Проведите финальное тестирование.**](#step4)

## Описание данных

- `RowNumber` — индекс строки в данных;
- `CustomerId` — уникальный идентификатор клиента;
- `Surname` — фамилия;
- `CreditScore` — кредитный рейтинг;
- `Geography` — страна проживания;
- `Gender` — пол;
- `Age` — возраст;
- `Tenure` — количество недвижимости у клиента;
- `Balance` — баланс на счёте;
- `NumOfProducts` — количество продуктов банка, используемых клиентом;
- `HasCrCard` — наличие кредитной карты;
- `IsActiveMember` — активность клиента;
- `EstimatedSalary` — предполагаемая зарплата;
- `Exited` (целевой признак) — факт ухода клиента.

## <a name="step1"></a>Шаг 1. Загрузите и подготовьте данные

Для начала сделаем импорт необходимых библиотек.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

Откроем файл с данными, запишем его в переменную `churn`.

In [2]:
try:
    churn = pd.read_csv('/Users/andreykol/Desktop/yandex_projects/project_Supervised_Learning/Churn.csv')
    # churn = pd.read_csv('datasets/Churn.csv')
except:
    print('Ошибка при чтении файла!')

Теперь подробнее изучим информацию в таблице.

In [3]:
print(churn.info())
print(churn.columns)
display(churn.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None
Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender',

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Итак, необходимо сделать предобработку данных. Будем действовать по следующему плану:
- удалим столбец `RowNumber`, который является излишним;
- заполним пропуски в столбце `Tenure`;
- закодируем категориальные переменные;
- нормализуем количественные переменные.

Итак, удалим столбец, отвечающий за индекс строки в данных.

In [4]:
churn = churn.drop('RowNumber', axis=1)

Узнаем, какие значения вообще есть в столбце, в котором записана информация о количестве недвижимости у клиента.

In [5]:
churn['Tenure'].value_counts(dropna=False)

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

По смыслу данных в столбце, вполне логично заполнить пропуски значением `0` (нет недвижимости).

In [6]:
churn['Tenure'] = churn['Tenure'].fillna(0)

Теперь произведём кодировку категориальных признаков в наших данных. Такими признаками являются `Geography` и `Gender`.

In [7]:
cat_cols = ['Geography', 'Gender']
trans_categ = pd.get_dummies(churn[cat_cols], drop_first=True) # новые столбцы
churn = churn.drop(labels=cat_cols, axis=1).join(trans_categ) # удалим старые, вставим новые

Переходим к нормализации признаков, а именно столбцов `CreditScore`, `Age`, `Tenure`, `Balance`, `EstimatedSalary`. Мы будем использовать инструмент `StandardScaler` для упрощения, предполагая, что распределение данных нормально (это, однако, совсем не так).

In [8]:
num_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
churn[num_cols] = scaler.fit_transform(churn[num_cols])

Итак, посмотрим, какие данные мы имеем на этот момент.

In [9]:
churn.head(10)

,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,15634602,Hargrave,-0.326221,0.293517,-0.817441,-1.225848,1,1,1,0.021886,1,0,0,0
1,15647311,Hill,-0.440036,0.198164,-1.138838,0.117350,1,0,1,0.216534,0,0,1,0
2,15619304,Onio,-1.536794,0.293517,1.110941,1.333053,3,1,0,0.240687,1,0,0,0
3,15701354,Boni,0.501521,0.007457,-1.138838,-1.225848,2,0,0,-0.108918,0,0,0,0
4,15737888,Mitchell,2.063884,0.388871,-0.817441,0.785728,1,1,1,-0.365276,0,0,1,0
5,15574012,Chu,-0.057205,0.484225,1.110941,0.597329,2,1,0,0.863650,1,0,1,1
6,15592531,Bartlett,1.774174,1.056346,0.789544,-1.225848,2,1,1,-1.565487,0,0,0,1
7,15656148,Obinna,-2.840488,-0.946079,-0.174647,0.618019,4,1,0,0.334854,1,1,0,0
8,15792365,He,-1.547141,0.484225,-0.174647,1.050820,2,0,1,-0.437329,0,0,0,1
9,15592389,H?,0.346319,-1.136786,-0.817441,0.931463,1,1,1,-0.493230,0,0,0,1


Теперь разделим нашу выборку на обучающую, валидационную и тестовую, а также отделим целевой признак от всех остальных.

In [10]:
X = churn.drop(['CustomerId', 'Surname', 'Exited'], axis=1) # ясно, что фамилия и id -- ненужные для модели признаки
y = churn['Exited']
X_nottest, X_test, y_nottest, y_test = train_test_split(X, y, 
                                                        test_size=0.2, random_state=42, shuffle=True)
X_train, X_valid, y_train, y_valid = train_test_split(X_nottest, y_nottest, 
                                                      test_size=0.25, random_state=42, shuffle=True)

In [11]:
print(X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape)

(6000, 11) (2000, 11) (2000, 11) (6000,) (2000,) (2000,)


Итак, данные готовы для подачи на вход модели.

## <a name="step2"></a>Шаг 2. Исследуйте баланс классов, обучите модель без учёта дисбаланса

Итак, для того, чтобы исследовать баланс классов, достаточно узнать, в каком количестве у нас имеются значения `0` и `1` в `target` (в известной части столбца, то есть `y_nottest`).

In [12]:
y_nottest.value_counts()

0    6356
1    1644
Name: Exited, dtype: int64

Мы видим, что размеры классов относятся примерно как 4:1 в пользу неушедших клиентов. Будем принимать дисбаланс классов в расчёт на следующем шаге. А пока обучим модель. Перед нами стоит задача классификации. Для её решения будем использовать модели логистической регрессии и случайного леса (с вариацией гиперпараметров).

Итак, начнём с логистической регрессии.

In [13]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
y_valid_pred = model.predict(X_valid)
y_train_pred = model.predict(X_train)
print('F1-score for logistic regression (TRAIN):', f1_score(y_train, y_train_pred))
print('F1-score for logistic regression (VALID):', f1_score(y_valid, y_valid_pred))
print('Area under the ROC curve for logistic regression:', roc_auc_score(y_valid, y_valid_pred_probs))

F1-score for logistic regression (TRAIN): 0.31470230862697446
F1-score for logistic regression (VALID): 0.3090909090909091
Area under the ROC curve for logistic regression: 0.7835256495344285


Довольно неудачный показатель F1-меры у нас получился, всего лишь 0.309, площадь под ROC-кривой равна примерно 0.78. Перейдём к настройке наилучшего случайного леса. Будем варьировать два гиперпараметра — количество деревьев (от 10 до 100 с шагом 10) и максимальную глубину дерева (от 1 до 15). Если перебирать значения в двойном цикле, нужно было бы оценить 150 моделей. Для ускорения процесса сначала, фиксируя количество деревьев в лесу на уровне 20, найдём оптимальную глубину, а затем, при помощи найденной глубины найдём оптимальное количество деревьев (всего 25 моделей).

In [14]:
best_f1 = 0
best_f1_train = 0
best_aucroc = 0
best_depth = 0
for depth in range(1, 16):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=42)
    model.fit(X_train, y_train)
    y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
    y_valid_pred = model.predict(X_valid)
    y_train_pred = model.predict(X_train)
    f1 = f1_score(y_valid, y_valid_pred)
    f1_train = f1_score(y_train, y_train_pred)
    aucroc = roc_auc_score(y_valid, y_valid_pred_probs)
    if f1 > best_f1:
        best_f1 = f1
        best_f1_train = f1_train
        best_aucroc = aucroc
        best_depth = depth
print('Best maximum depth:', best_depth)
print('F1-score for best random forest (TRAIN):', best_f1_train)
print('F1-score for best random forest (VALID):', best_f1)
print('Area under the ROC curve for best random forest:', best_aucroc)

Best maximum depth: 13
F1-score for best random forest (TRAIN): 0.8587155963302752
F1-score for best random forest (VALID): 0.6039453717754173
Area under the ROC curve for best random forest: 0.8511208655068192


Все целевые показатели стали выше. Получили, что оптимальная максимальная глубина деревьев равна 13. Мы уже близки к нашей цели, ведь F1-мера у нас стала равна 0.603. Но пока мы не знаем, какой она будет на тестовой выборке. Двигаемся далее.

In [15]:
best_f1 = 0
best_f1_train = 0
best_aucroc = 0
best_ests = 0
for ests in range(10, 110, 10):
    model = RandomForestClassifier(n_estimators=ests, max_depth=best_depth, random_state=42)
    model.fit(X_train, y_train)
    y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
    y_valid_pred = model.predict(X_valid)
    y_train_pred = model.predict(X_train)
    f1 = f1_score(y_valid, y_valid_pred)
    f1_train = f1_score(y_train, y_train_pred)
    aucroc = roc_auc_score(y_valid, y_valid_pred_probs)
    if f1 > best_f1:
        best_f1 = f1
        best_f1_train = f1_train
        best_aucroc = aucroc
        best_ests = ests
print('Best number of estimators:', best_ests)
print('F1-score for best random forest (TRAIN):', best_f1_train)
print('F1-score for best random forest (VALID):', best_f1)
print('Area under the ROC curve for best random forest:', best_aucroc)

Best number of estimators: 20
F1-score for best random forest (TRAIN): 0.8587155963302752
F1-score for best random forest (VALID): 0.6039453717754173
Area under the ROC curve for best random forest: 0.8511208655068192


Таким образом, мы можем сказать, что при дисбалансе классов (и неучитывании его моделью) логистическая регрессия справляется намного хуже (F1-мера чуть более 0.3), чем почти любая модель случайного леса. При этом, анализ показал, что наилучшее количество деревьев в лесу — 20, а наилучшая максимальная глубина — 13. На валидационной выборке мы достигли показателя F1-меры, большего 0.6, а площадь под ROC-кривой оказалась чуть больше 0.851.

## <a name="step3"></a>Шаг 3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую

На прошлом шаге мы выяснили, что имеет место сильный дисбаланс классов. Теперь будем тестировать те же самые модели, что и раньше, но теперь учитывая дисбаланс классов в каждой из них. Мы можем сделать это разными способами, например, апсэмплингом или даунсэмплингом. В данной работе поступим иначе. И у `LinearRegression()`, и у `RandomForestClassifier()` есть параметр, который мы будем использовать далее — `class_weight='balanced'`. Он придаст больший вес объектам с ответом `1`, причём ровно во столько раз, во сколько этот класс меньше другого (около 4 в нашем случае).

Снова начинаем с логистической регрессии.

In [16]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
y_valid_pred = model.predict(X_valid)
y_train_pred = model.predict(X_train)
print('F1-score for logistic regression (TRAIN, BALANCED):', f1_score(y_train, y_train_pred))
print('F1-score for logistic regression (VALID, BALANCED):', f1_score(y_valid, y_valid_pred))
print('Area under the ROC curve for logistic regression (BALANCED):', roc_auc_score(y_valid, y_valid_pred_probs))

F1-score for logistic regression (TRAIN, BALANCED): 0.4883855981416957
F1-score for logistic regression (VALID, BALANCED): 0.5172413793103448
Area under the ROC curve for logistic regression (BALANCED): 0.7847096032992031


Мы наблюдаем значительное улучшение результатов для логистической регрессии после устранения дисбаланса классов. Однако, F1-мера на валидационной выборке по-прежнему ниже приемлемого уровня, она чуть превосходит 0.517. Площадь под ROC-кривой около 0.784. Руководствуясь принципами, выбранными ранее, теперь будем подбирать лучшую модель среди случайных лесов. Начнём с подбора глубина деревьев при фиксированном их числе, а затем узнаем наилучшее количество деревьев в лесу.

In [17]:
best_f1 = 0
best_f1_train = 0
best_aucroc = 0
best_depth = 0
for depth in range(1, 16):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)
    y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
    y_valid_pred = model.predict(X_valid)
    y_train_pred = model.predict(X_train)
    f1 = f1_score(y_valid, y_valid_pred)
    f1_train = f1_score(y_train, y_train_pred)
    aucroc = roc_auc_score(y_valid, y_valid_pred_probs)
    if f1 > best_f1:
        best_f1 = f1
        best_f1_train = f1_train
        best_aucroc = aucroc
        best_depth = depth
print('Best maximum depth:', best_depth)
print('F1-score for best random forest (TRAIN, BALANCED):', best_f1_train)
print('F1-score for best random forest (VALID, BALANCED):', best_f1)
print('Area under the ROC curve for best random forest (BALANCED):', best_aucroc)

Best maximum depth: 8
F1-score for best random forest (TRAIN, BALANCED): 0.694683908045977
F1-score for best random forest (VALID, BALANCED): 0.6351931330472103
Area under the ROC curve for best random forest (BALANCED): 0.8717645335664623


Итак, на валидационной выборке мы достигли уровня F1-меры в 0.635, что хорошо. Заметим, что оптимальная максимальная глубина деревьев теперь равна всего лишь 8. Далее узнаем, как это отразится на подборе количества деревьев в лесу ниже.

In [18]:
best_f1 = 0
best_f1_train = 0
best_aucroc = 0
best_ests = 0
for ests in range(10, 110, 10):
    model = RandomForestClassifier(n_estimators=ests, max_depth=best_depth, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)
    y_valid_pred_probs = model.predict_proba(X_valid)[:, 1]
    y_valid_pred = model.predict(X_valid)
    y_train_pred = model.predict(X_train)
    f1 = f1_score(y_valid, y_valid_pred)
    f1_train = f1_score(y_train, y_train_pred)
    aucroc = roc_auc_score(y_valid, y_valid_pred_probs)
    if f1 > best_f1:
        best_f1 = f1
        best_f1_train = f1_train
        best_aucroc = aucroc
        best_ests = ests
print('Best number of estimators:', best_ests)
print('F1-score for best random forest (TRAIN):', best_f1_train)
print('F1-score for best random forest (VALID):', best_f1)
print('Area under the ROC curve for best random forest:', best_aucroc)

Best number of estimators: 20
F1-score for best random forest (TRAIN): 0.694683908045977
F1-score for best random forest (VALID): 0.6351931330472103
Area under the ROC curve for best random forest: 0.8717645335664623


Итак, мы получили чуть улучшенные результаты, F1-мера на валидационной выборке достигла 0.635, а площадь под ROC-кривой теперь равна 0.871. Лучшей моделью становится случайный лес с 20 деревьями максимальной глубины 8 и балансировкой классов.

## <a name="step4"></a>Шаг 4. Проведите финальное тестирование

До этого момента мы никак не использовали ответы тестовой выборки (`y_test`), они в реальности нам неизвестны. Теперь мы предскажем эти значения по `X_test`. Мы ожидаем увидеть примерно те же значения F1-меры и площади под ROC-кривой, что и на валидационной выборке. Теперь мы можем обучать модель на совокупности обучающей и валидационной выборки (`X_nottest` и `y_nottest`).

In [21]:
%%time

model = RandomForestClassifier(n_estimators=best_ests, max_depth=best_depth, random_state=42, class_weight='balanced')
model.fit(X_nottest, y_nottest)
y_test_pred_probs = model.predict_proba(X_test)[:, 1]
y_test_pred = model.predict(X_test)
print('F1-score for the best model (TEST):', f1_score(y_test, y_test_pred))
print('Accuracy for the best model (TEST):', accuracy_score(y_test, y_test_pred))
print('Precision for the best model (TEST):', precision_score(y_test, y_test_pred))
print('Recall for the best model (TEST):', recall_score(y_test, y_test_pred))
print('Area under the ROC curve for the best model:', roc_auc_score(y_test, y_test_pred_probs))

F1-score for the best model (TEST): 0.6163793103448275
Accuracy for the best model (TEST): 0.822
Precision for the best model (TEST): 0.5345794392523364
Recall for the best model (TEST): 0.727735368956743
Area under the ROC curve for the best model: 0.86251862478248
CPU times: user 217 ms, sys: 6.55 ms, total: 224 ms
Wall time: 226 ms


Да, значение F1-меры превысило 0.616, основная задача нами выполнена, мы достигли необходимого порога в 0.59 на тестовой выборке. Значение площади под ROC-кривой в данном случае примерно равно 0.862. Для более цельной картины мы подсчитали и другие известные нам метрики.

Таким образом, мы построили предсказательную модель, которая удовлетворяет требованиям задачи. Время её работы не превосходит 1 секунды, что является отличным результатом.